Реализовать алгоритмы TF-IDF + word2vec (чтобы можно было складывать и вычитать слова). Также необходимо найти датасет для обучения написанных алгоритмов)

In [ ]:
pip install gensim scikit-learn nltk

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [2]:
import gensim
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import re
from nltk.corpus import stopwords

# 🔹 Загружаем необходимые данные NLTK
nltk.download("stopwords")
nltk.download("punkt")

# 🔹 Указанный путь к файлу
dataset_path = r"C:\Users\asus\Desktop\ML\dataset\text8"

# 🔹 Чтение текста из файла с обработкой ошибок
with open(dataset_path, "r", encoding="utf-8", errors="ignore") as f:
    text = f.read()

# 🔹 Альтернативная токенизация (выбираем нужный метод)
def tokenize_text(text, method="regex"):
    text = text.lower()  # Приводим к нижнему регистру

    if method == "split":
        return text.split()  # Простая токенизация (разделение по пробелу)

    elif method == "regex":
        return re.findall(r"\b\w+\b", text)  # Удаляем знаки препинания и оставляем слова

    else:
        raise ValueError("Неподдерживаемый метод токенизации")

# Выбираем метод токенизации
print("Токенизация текста...")
tokenized_corpus = [tokenize_text(text, method="regex")]  # Вместо word_tokenize()

# 🔹 Обучение Word2Vec
print("Обучение Word2Vec...")
word2vec_model = gensim.models.Word2Vec(
    sentences=tokenized_corpus, vector_size=100, window=5, min_count=5, workers=4
)

# 🔹 Разделяем текст на предложения для TF-IDF
sentences = text.lower().split(".")[:10000]  # Берем первые 10 000 предложений

# 🔹 TF-IDF векторизация
print("Вычисление TF-IDF...")
vectorizer = TfidfVectorizer(stop_words=stopwords.words("english"))
tfidf_matrix = vectorizer.fit_transform(sentences)
feature_names = vectorizer.get_feature_names_out()

# 🔹 Функция для сложения и вычитания слов
def word_arithmetic(word1, word2, operation="+"):
    try:
        if operation == "+":
            result_vector = word2vec_model.wv[word1] + word2vec_model.wv[word2]
        elif operation == "-":
            result_vector = word2vec_model.wv[word1] - word2vec_model.wv[word2]
        else:
            return "Некорректная операция"
        
        similar_words = word2vec_model.wv.similar_by_vector(result_vector, topn=5)
        return similar_words
    except KeyError as e:
        return f"Слово {str(e)} отсутствует в модели"

# 🔹 Примеры работы
print(word_arithmetic("king", "man", "-"))  # Должно дать что-то типа "queen"
print(word_arithmetic("paris", "france", "+"))  # Может дать "capital"


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Токенизация текста...
Обучение Word2Vec...
Вычисление TF-IDF...
[('souffle', 0.394724577665329), ('greenway', 0.39336642622947693), ('unction', 0.3841506540775299), ('penal', 0.37015044689178467), ('feudal', 0.3689629137516022)]
[('paris', 0.8194121718406677), ('france', 0.777579665184021), ('believe', 0.581599771976471), ('property', 0.5730543732643127), ('world', 0.5716093182563782)]


Этот вывод состоит из двух частей, которые описывают результаты обработки текста с использованием двух разных методов: **TF-IDF** и **Word2Vec**.

### 1. **Результаты TF-IDF**
Первая строка с результатами относится к вычислениям **TF-IDF** (Term Frequency-Inverse Document Frequency), что представляет собой метод для оценки важности слов в документе с учетом частоты их появления в других документах (или в контексте всего корпуса).

Токены (слова) в выводе с их соответствующими значениями TF-IDF:

```python
[('souffle', 0.394724577665329), 
 ('greenway', 0.39336642622947693), 
 ('unction', 0.3841506540775299), 
 ('penal', 0.37015044689178467), 
 ('feudal', 0.3689629137516022)]
```

Здесь, каждое слово представлено вместе с числовым значением, которое говорит о его важности в текущем контексте. Важно понимать:

- **TF (Term Frequency)** — это частота, с которой слово встречается в документе.
- **IDF (Inverse Document Frequency)** — это мера того, насколько слово редко встречается в других документах, и чем реже слово встречается, тем более важным оно становится в документе.

Чем выше значение TF-IDF, тем более значимым считается слово в контексте данного документа (относительно других документов в корпусе).

- Например, слово **"souffle"** имеет высокое значение TF-IDF — 0.3947, что указывает на его высокую значимость в тексте.
- Слова вроде **"greenway"**, **"unction"**, **"penal"** и **"feudal"** также имеют значительные значения, что свидетельствует о их важности.

### 2. **Результаты Word2Vec**
Вторая часть вывода относится к **Word2Vec**, который является методом для получения векторных представлений слов, позволяющих вычислять их "схожесть" и выполнять операции, такие как сложение или вычитание слов (например, "король" - "мужчина" + "женщина" = "королева").

Результаты с использованием **Word2Vec**:

```python
[('paris', 0.8194121718406677), 
 ('france', 0.777579665184021), 
 ('believe', 0.581599771976471), 
 ('property', 0.5730543732643127), 
 ('world', 0.5716093182563782)]
```

Здесь слова **"paris"**, **"france"**, **"believe"**, **"property"**, **"world"** являются самыми близкими по смыслу или контексту к словам в тексте, и числа рядом с ними указывают на степень "схожести" между ними. Чем выше значение (ближе к 1), тем более схожими являются слова в векторном пространстве.

- Например, слова **"paris"** и **"france"** имеют высокое значение схожести (0.819 и 0.777), что не удивительно, потому что "Париж" и "Франция" тесно связаны по контексту.
- Также можно заметить, что **"believe"**, **"property"** и **"world"** имеют довольно высокие значения схожести, что говорит о их важности или близости к остальному контексту.

### В заключение
- **TF-IDF** показывает, какие слова важны для данного текста относительно других документов.
- **Word2Vec** помогает найти слова, схожие по смыслу в контексте векторных представлений.

Оба метода дают полезную информацию о текстах и их содержании.